# Feature extraction

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import feature_extraction

## Load dataset

In [3]:
# Load CB16 dataset
df = pd.read_json("../data/clickbait17-train-170331/instances.jsonl", lines=True, encoding='utf8')
df.set_index("id", inplace=True)
df_raw = df.copy()

df_truth = pd.read_json("../data/clickbait17-train-170331/truth.jsonl", lines=True, encoding='utf8')
df_truth.set_index("id", inplace=True)
df_truth_raw = df_truth.copy()

In [4]:
# Add truth judgments to df
df = df.join(df_truth[["truthClass"]]) # also available: truthClass, truthJudgments, truthMean, truthMedian, truthMode

In [5]:
df.head(1)

,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,truthClass
id,,,,,,,,,
608310377143799808,[],[Apple's iOS 9 'App thinning' feature will giv...,Tue Jun 09 16:31:10 +0000 2015,['App thinning' will be supported on Apple's i...,'App thinning' will be supported on Apple's iO...,"Apple,gives,gigabytes,iOS,9,app,thinning,featu...",[Paying for a 64GB phone only to discover that...,Apple gives back gigabytes: iOS 9 'app thinnin...,no-clickbait


## Extract features

In [6]:
fe = feature_extraction.FeatureExtractor(df.head(10))
labels, features = fe.extract_features()

In [7]:
display(features)

,numChars_PostTitle,ratioChars_PostImagePostTitle,diffChars_PostTitleArticleKeywords,diffChars_PostTitlePostImage,ratioWords_PostImagePostTitle,numWords_PostTitle,ratioWords_ArticleDescPostTitle,ratioChars_ArticleDescPostTitle,ratioChars_ArticleTitlePostTitle,ratioWords_ArticleTitlePostTitle,diffWords_PostTitleArticleKeywords,ratioChars_ArticleParagraphsArticleDesc
id,,,,,,,,,,,,
608310377143799808,63.0,0.015873,15.0,64.0,0.076923,13.0,18.0,2.444444,1.380952,1.307692,0.0,0.644958
609297109095972864,85.0,0.011765,35.0,86.0,0.066667,15.0,12.0,1.647059,0.552941,0.533333,4.0,0.866667
609504474621612032,78.0,0.012821,78.0,79.0,0.083333,12.0,10.0,1.692308,0.730769,0.833333,12.0,1.611269
609748367049105408,52.0,0.019231,62.0,53.0,0.090909,11.0,6.0,1.884615,1.000000,1.000000,7.0,2.230214
608688782821453824,76.0,0.013158,18.0,77.0,0.062500,16.0,6.0,1.355263,0.381579,0.375000,5.0,2.209894
609551038983475200,54.0,0.018519,54.0,55.0,0.090909,11.0,11.0,-1.000000,1.018519,1.000000,11.0,-1.000000
609447408955719680,77.0,0.012987,20.0,78.0,0.062500,16.0,10.0,1.597403,0.870130,0.625000,6.0,1.282192
609027430624288768,57.0,0.017544,57.0,58.0,0.090909,11.0,18.0,2.508772,0.982456,1.000000,11.0,0.879720
608229011572068352,70.0,0.014286,89.0,71.0,0.066667,15.0,11.0,1.642857,0.742857,0.800000,5.0,2.365217
